In [1]:
%load_ext Cython

In [2]:
%%cython

from libc.stdint cimport *

cdef extern from "Python.h":
    object PyString_FromStringAndSize(char *, Py_ssize_t)
    char *PyBytes_AsString(object)

cdef extern from "/Users/bob/code/podcast/rss_extract/lib/histogram/murmur3.h":
    cdef void MurmurHash3_x64_128(const void * key, const int _len, uint32_t seed, void * out)
    
def murmur_hash(bytes s):
    cdef Py_ssize_t length = len(s)
    cdef char *sb = PyBytes_AsString(s)
    cdef uint64_t[2] res
    
    MurmurHash3_x64_128(sb, length, 42, res)
    
    return res[0] ^ res[1]
    
    

In [3]:
murmur_hash("test".encode('utf-8'))

14466693972460839356

In [38]:
import re
entity_re = re.compile(r'\\u([0-9a-f]{4})')
def decode_entities(v):
    return entity_re.sub(lambda match: chr(int(match.group(1), 16)), v)

In [27]:
? chr

In [4]:
import pickle

In [5]:
import numpy as np

In [42]:
with open('./token_dists.pickle', 'rb') as inf:
    dists = pickle.load(inf)

In [41]:
def fix_keys(dist):
    res = {}
    for k, v in dist.items():
        res[decode_entities(k)] = v
    return res

In [43]:
dists = [fix_keys(v) for v in dists if len(v) > 0]

In [44]:
len(dists)

1392

In [62]:
dists_lowercase = []
for dist in dists:
    v = {}
    for k, c in dist.items():
        k = k.lower()
        if k in v:
            v[k] += c
        else:
            v[k] = c
    dists_lowercase.append(v)

In [63]:
dists = dists_lowercase

In [64]:
dist_hashes = []
for dist in dists:
    dlen = len(dist)
    hash_arr = np.empty((dlen,), dtype=np.uint64)
    count_arr = np.zeros((dlen,), dtype=np.uint64)
    for idx, (k, c) in enumerate(dist.items()):
        h = murmur_hash(k.encode('utf-8'))
        hash_arr[idx] = h
        count_arr[idx] = c
    dist_hashes.append((hash_arr, count_arr))

In [65]:
dist_hashes[0]

(array([ 6542722928290085166, 11955699455548345443,  6921773163938638153,
        ...,  9039886265152516753,   561218049625248724,
        12685494650715651576], dtype=uint64),
 array([1800, 9761, 4822, ...,    1,    1,    1], dtype=uint64))

In [66]:
sorted_dist_hashes = []
for h, c in dist_hashes:
    idxes = np.argsort(h)
    sorted_dist_hashes.append((h[idxes], c[idxes]))

In [67]:
sorted_dist_hashes[1]

(array([   72070893150096027,   139428885052208584,   140570187466950067,
          147530717713822329,   159234690102437018,   163954680460945382,
          177046348584101622,   227446426996025206,   237188749443315877,
          276287089699395670,   325354432707452096,   386361036130087665,
          428938599612749114,   451521113658592875,   465474334209975165,
          507229187383669329,   545121197812336463,   550120577644887867,
          551114696100349979,   561977573161684396,   562056088709496250,
          568786754235137239,   667474728443831911,   693911418889159014,
          706066105445191250,   707984191652811888,   812049558892549454,
          854498940094406552,   867617679497147089,   978838221826738503,
          983711964283762621,   994827356953513381,  1023335688996444883,
         1041471357125997668,  1043919380356768157,  1066840804797798782,
         1070461328702153130,  1073030030509327534,  1078386024212994099,
         1106858698018407619,  1228670

In [68]:
set([np.argmin(v[0]) for v in sorted_dist_hashes])

{0}

In [69]:
lengths = np.array([v[0].shape[0] for v in sorted_dist_hashes])

In [70]:
lengths

array([65763,   546, 16927, ...,   316,  1295,   664])

In [71]:
res = np.empty((1 + lengths.shape[0] + np.sum(lengths)*2,), dtype=np.uint64)

In [72]:
res[0] = lengths.shape[0]

In [73]:
res[1:(1 + lengths.shape[0])] = lengths

In [74]:
offset = 1 + lengths.shape[0]
for h, c in sorted_dist_hashes:
    hs = h.shape[0]
    cs = c.shape[0]
    res[offset:(offset + hs)] = h
    offset += hs
    res[offset:(offset + cs)] = c
    offset += cs

In [75]:
res[-1000:-990]

array([9419583467273265819, 9494206043694450962, 9540537239493949058,
       9546083669112437070, 9550374232551791872, 9564686715978789561,
       9568606877582353554, 9572120805474395499, 9582261194407524846,
       9591347944122404375], dtype=uint64)

In [76]:
res.tofile('token_dists.bin')

In [77]:
!ls -lh token_dists.bin

-rw-r--r--  1 bob  staff   437M Apr 15 14:09 token_dists.bin


In [106]:
res[0]

1392

In [79]:
res[-712:-700]

array([17099400986748975806, 17126617102872912932, 17205451217589082872,
       17217264110123707600, 17239752794746872376, 17258088682278238853,
       17293087393332838770, 17310626912190217444, 17316995750203367552,
       17332563090086013606, 17336924912161307323, 17374192831438801270],
      dtype=uint64)

In [82]:
test_values = []
for dist in dists:
    v = []
    for idx, it in enumerate(dist.items()):
        if it[1] > 100:
            v.append(it)
        if len(v) > 20:
            break
    test_values.append(v)

In [87]:
test_values = list(zip(range(len(test_values)), test_values))

In [121]:
for c, x in test_values[-10:]:
    if len(x) < 1: continue
    print("(%d, [%s]);" % (c, ';'.join('("%s", %d)' % (''.join('\\x%s' % hex(char).replace('0x', '') for char in v[0].encode('utf-8')),v[1]) for v in x)))

(1382, [("\x61\x6d\x61\x74\x65\x75\x72", 125);("\xe4\xb8\x8a\xe4\xb8\x80\xe4\xb8\x96\xef\xbc\x8c\xe8\x8b\x8d\xe5\x9b\xbd\xe4\xb8\x9e\xe7\x9b\xb8\xe4\xb9\x8b\xe5\xa5\xb3\xe4\xb8\x8a\xe5\xae\x98\xe6\xb2\xab\xef\xbc\x8c\xe7\xbb\x9d\xe7\xbe\x8e\xe5\x80\xbe\xe5\x9f\x8e\xef\xbc\x8c\xe6\x80\xa7\xe5\xad\x90\xe5\x8d\xb4\xe5\xa4\xaa\xe8\xbf\x87\xe8\xbd\xaf\xe5\xbc\xb1\xef\xbc\x8c\xe8\xa2\xab\xe5\xad\xaa\xe7\x94\x9f\xe5\xa6\xb9\xe5\xa6\xb9\xe6\x8a\xa2\xe5\xb0\xbd\xe4\xba\x86\xe6\x89\x80\xe6\x9c\x89\xe9\xa3\x8e\xe5\xa4\xb4\xef\xbc\x8c\xe5\x8d\xb4\xe5\x9c\xa8\xe9\x93\xb6\xe6\x9c\x88\xe5\x9b\xbd\xe8\xa6\x81\xe6\xb1\x82\xe8\x81\x94\xe5\xa7\xbb\xe4\xb9\x8b\xe6\x97\xb6\xef\xbc\x8c\xe8\xa2\xab\xe4\xba\xb2\xe4\xba\xba\xe6\xaf\xab\xe4\xb8\x8d\xe7\x95\x99\xe6\x83\x85\xe5\x9c\xb0\xe6\x8e\xa8\xe4\xba\x86\xe5\x87\xba\xe5\x8e\xbb\xef\xbc\x8c\xe5\x8f\xaa\xe4\xb8\xba\xe4\xbf\x9d\xe4\xbd\x8f\xe5\xa5\xb9\xe9\x82\xa3\xe4\xb8\xaa\xe5\xa6\x82\xe7\x8f\xa0\xe5\xa6\x82\xe5\xae\x9d\xe7\x9a\x84\xe5\xa6\xb9\xe5\xa6\xb9\xe3

In [111]:
list(zip(range(100), [np.sum(v[1]) for v in sorted_dist_hashes][:100]))

[(0, 705623),
 (1, 1321),
 (2, 129561),
 (3, 392),
 (4, 47159),
 (5, 926708),
 (6, 73177),
 (7, 2202332),
 (8, 33018),
 (9, 2312),
 (10, 89386),
 (11, 21),
 (12, 108),
 (13, 15006),
 (14, 612),
 (15, 326067),
 (16, 10711),
 (17, 7680),
 (18, 12617),
 (19, 24666),
 (20, 91488),
 (21, 67381),
 (22, 87777),
 (23, 529),
 (24, 39184),
 (25, 889444),
 (26, 23638),
 (27, 1021532),
 (28, 1510),
 (29, 12101),
 (30, 20961),
 (31, 8578),
 (32, 1509),
 (33, 629),
 (34, 38939),
 (35, 127),
 (36, 5850),
 (37, 3125),
 (38, 12910),
 (39, 17945),
 (40, 45189),
 (41, 1959),
 (42, 6721),
 (43, 7723),
 (44, 2489),
 (45, 11609),
 (46, 883431),
 (47, 116486),
 (48, 3951997),
 (49, 568776),
 (50, 4884),
 (51, 1606),
 (52, 37133),
 (53, 17438),
 (54, 5812),
 (55, 751943),
 (56, 9716),
 (57, 2762318),
 (58, 184930),
 (59, 3849845),
 (60, 15),
 (61, 8309),
 (62, 2421),
 (63, 392),
 (64, 313417),
 (65, 35599),
 (66, 9093),
 (67, 10024),
 (68, 128859),
 (69, 1463),
 (70, 402937),
 (71, 9402),
 (72, 8789),
 (73, 2

In [109]:
l = 28644063
r = 458137848

In [110]:
r / l

15.994164235709158

In [113]:
murmur_hash("die".encode('utf-8')) == 3485591795621796808

True

In [114]:
4125 / 740

5.574324324324325

In [116]:
sorted_dist_hashes[18][0].shape

(4125,)

In [117]:
len(sorted_dist_hashes)

1392